In [28]:
#importando as bibliotecas
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Parte 1

In [2]:
#Abrindo o webdrive com o endereço da url
url = 'https://1xbet.whoscored.com/Statistics'
driver = webdriver.Chrome()
driver.get(url = url)

In [3]:
#lista com todos os links das tabela de estatistica dos times
list_link_statistics = ["//a[contains(@href,'#top-team-stats-offensive')]",
                        "//a[contains(@href,'#top-team-stats-defensive')]",                        
                        "//a[contains(@href,'#top-team-stats-detailed')]",
                        "//a[contains(@href,'#top-team-stats-summary')]"
                       ]
#percorre a lista para a carregar o DOM
for link in list_link_statistics:
    driver.find_element(By.XPATH,link).location_once_scrolled_into_view
    driver.find_element(By.XPATH,link).click()
    time.sleep(1)

In [4]:
#função que faz a leitura das tabelas
def Read_Table_Statistics(driver,tbl):
    html = driver.page_source
    Beautiful_Soup = BeautifulSoup(html, 'html.parser')
    tabela = str(Beautiful_Soup.find_all('table', {'id': tbl}))
    df = pd.read_html(tabela)

    return df,Beautiful_Soup

In [5]:
#função pega as informações do cartão(Yellow/Red)
def Set_Card(B_Soup,color_card):
    card_box = color_card+'-card-box'
    card = list(B_Soup.find_all('span',{'class':card_box}))
    card = [str(i) for i in card]    
    card = [int(i.split("</span>")[0].split("<span class=\""+card_box+"\">")[-1]) for i in card] 
    
    return card

In [6]:
#declaração de variaveis
index = 0
tpages = driver.find_elements(By.ID,"totalPages")[index]
df_Summary = pd.DataFrame()
df_Offensive = pd.DataFrame()
df_Defensive = pd.DataFrame()
df_Detailed = pd.DataFrame()
red_card = []
yellow_card = []
wait = WebDriverWait(driver,timeout=10)
#loop que vai mudar de pagina
for page in range(1,int(tpages.get_attribute(name='value'))):
    #leitura das tabelas
    df_table,soup = Read_Table_Statistics(driver,'top-team-stats-summary-grid')  
    #leitura dos cartões
    red_aux = Set_Card(soup,"red")    
    yellow_aux = Set_Card(soup,"yellow")    
    #concatenando os dataframe
    df_Summary = pd.concat([df_Summary,df_table[0]],ignore_index=True)
    df_Offensive = pd.concat([df_Offensive,df_table[1]],ignore_index=True)
    df_Defensive = pd.concat([df_Defensive,df_table[2]],ignore_index=True)
    df_Detailed = pd.concat([df_Detailed,df_table[3]],ignore_index=True)
    #concatenando as lista dos cartões
    yellow_card = yellow_card + yellow_aux
    red_card = red_card + red_aux
    #clicando no link next para ir para a proxima pagina
    button_next = driver.find_elements(By.CSS_SELECTOR,"a#next.option.clickable")[index]
    wait.until(EC.element_to_be_clickable((button_next)))        
    driver.find_elements(By.CSS_SELECTOR,"a#next.option.clickable")[index].click()
    time.sleep(2)
#leitura das tabelas    
df_table,soup = Read_Table_Statistics(driver,'top-team-stats-summary-grid')
#concatenando os dataframe
df_Summary = pd.concat([df_Summary,df_table[0]],ignore_index=True)
df_Offensive = pd.concat([df_Offensive,df_table[1]],ignore_index=True)
df_Defensive = pd.concat([df_Defensive,df_table[2]],ignore_index=True)
df_Detailed = pd.concat([df_Detailed,df_table[3]],ignore_index=True)
#leitura dos cartões
red_aux = Set_Card(soup,"red")    
yellow_aux = Set_Card(soup,"yellow") 
#concatenando as lista dos cartões
yellow_card = yellow_card + yellow_aux
red_card = red_card + red_aux
#adicionando os cartões no dataframe Summary e tirando a coluna Discipline
#Retirando o numero antes dos nomes na coluna Team
df_Summary["YellowCard"] = yellow_card
df_Summary['RedCard'] = red_card
df_Summary.drop(labels="Discipline",axis=1)
df_Summary["Team"] = df_Summary["Team"].str.split(". ",1,expand=True)[1]
driver.find_elements(By.CSS_SELECTOR,"a#prev.option.clickable")[0].click()
time.sleep(2)
driver.find_elements(By.CSS_SELECTOR,"a#first.option.clickable")[0].click()

In [7]:
df_Summary

,Team,Tournament,Goals,Shots pg,Discipline,Possession%,Pass%,AerialsWon,Rating,YellowCard,RedCard
0,Manchester City,Premier League,68,18.5,341,68.0,90.0,11.9,7.10,34,1
1,Liverpool,Premier League,73,19.5,341,62.5,84.4,15.8,7.08,34,1
2,Bayern Munich,Bundesliga,77,20.4,221,63.8,85.6,13.0,7.02,22,1
3,Chelsea,Premier League,57,14.2,481,59.9,86.3,14.6,6.94,48,1
4,Real Madrid,LaLiga,56,17.4,500,61.4,89.1,10.6,6.90,50,0
...,...,...,...,...,...,...,...,...,...,...,...
93,Levante,LaLiga,30,10.3,796,46.3,78.0,12.3,6.44,79,6
94,Norwich,Premier League,18,9.8,451,42.7,77.5,13.6,6.42,45,1
95,Bordeaux,Ligue 1,38,11.3,643,46.8,80.9,12.0,6.42,64,3
96,Greuther Fuerth,Bundesliga,24,9.8,500,42.8,74.1,15.3,6.38,50,0


In [8]:
df_Offensive

,Team,Tournament,Shots pg,Shots OT pg,Dribbles pg,Fouled pg,Rating
0,1. Bayern Munich,Bundesliga,20.4,8.3,15.3,10.4,7.02
1,2. Paris Saint-Germain,Ligue 1,14.5,5.1,13.2,10.7,6.90
2,3. Lyon,Ligue 1,13.9,5.2,12.8,10.9,6.74
3,4. Barcelona,LaLiga,13.5,5.0,12.6,14.1,6.78
4,5. Bayer Leverkusen,Bundesliga,13.7,6.0,12.5,10.0,6.77
...,...,...,...,...,...,...,...
95,16. Borussia Dortmund,Bundesliga,14.3,5.6,10.8,11.6,6.81
96,17. Sassuolo,Serie A,15.1,5.7,10.7,12.9,6.61
97,18. Tottenham,Premier League,12.4,5.1,10.6,11.0,6.81
98,19. Mallorca,LaLiga,12.1,3.7,10.6,15.0,6.48


In [9]:
df_Defensive

,Team,Tournament,Shots pg,Tackles pg,Interceptions pg,Fouls pg,Offsides pg,Rating
0,1. Monaco,Ligue 1,10.0,20.1,10.8,13.1,2.4,6.65
1,2. Leeds,Premier League,15.3,19.8,10.6,12.0,1.8,6.52
2,3. Everton,Premier League,13.5,19.4,9.9,9.8,1.8,6.56
3,4. Genoa,Serie A,15.1,18.7,12.4,14.6,1.8,6.49
4,5. Saint-Etienne,Ligue 1,11.9,18.7,12.5,12.0,1.4,6.50
...,...,...,...,...,...,...,...,...
95,16. Wolverhampton Wanderers,Premier League,12.5,17.4,10.7,10.0,1.8,6.75
96,17. Lyon,Ligue 1,12.2,17.4,10.8,11.6,1.5,6.74
97,18. AC Milan,Serie A,10.6,17.3,9.8,12.6,2.0,6.76
98,19. Newcastle United,Premier League,14.5,17.3,9.7,10.5,1.5,6.59


In [10]:
df_Detailed

,Team,Tournament,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,1. Manchester City,Premier League,18.5,5.5,1.9,11.1,7.10
1,2. Liverpool,Premier League,19.5,6.2,1.9,11.4,7.08
2,3. Bayern Munich,Bundesliga,20.4,5.9,2.0,12.5,7.02
3,4. Chelsea,Premier League,14.2,5.0,1.0,8.2,6.94
4,5. Real Madrid,LaLiga,17.4,6.7,1.2,9.5,6.90
...,...,...,...,...,...,...,...
95,16. AC Milan,Serie A,15.0,6.2,0.8,8.1,6.76
96,17. Juventus,Serie A,13.9,5.0,0.9,7.9,6.76
97,18. Napoli,Serie A,15.8,6.4,1.3,8.0,6.76
98,19. Manchester United,Premier League,14.1,5.1,0.8,8.2,6.76


In [11]:
#lista com os link das tabelas de estatistica dos jogadores
list_link_player = ["//a[contains(@href,'#top-player-stats-defensive')]",
                    "//a[contains(@href,'#top-player-stats-offensive')]",
                    "//a[contains(@href,'#top-player-stats-passing')]",
                    "//a[contains(@href,'#top-player-stats-detailed')]",                        
                    "//a[contains(@href,'#top-player-stats-summary')]"
                   ]
#percorre a lista para a carregar o DOM
for link in list_link_player:
    driver.find_element(By.XPATH,link).location_once_scrolled_into_view
    driver.find_element(By.XPATH,link).click()
    time.sleep(1)

In [13]:
#declaração de variaveis
tpages = driver.find_elements(By.ID,"totalPages")
df_Player_Summary = pd.DataFrame()
df_Player_Offensive = pd.DataFrame()
df_Player_Defensive = pd.DataFrame()
df_Player_Passing = pd.DataFrame()
wait = WebDriverWait(driver,timeout=10)
#loop que vai percorrer todos os link da list_link_player
for item in range(4,8):
    #delcaração de um dataframe auxiliar
    df_Aux = pd.DataFrame()
    tpages = driver.find_elements(By.ID,"totalPages")
    #loop que vai mudar de pagina
    for page in range(1,int(tpages[item].get_attribute(name='value'))-1):
        #leitura das tabelas
        html = driver.page_source
        Beautiful_Soup = BeautifulSoup(html, 'html.parser')
        tabela = str(Beautiful_Soup.find_all('div', {'class': "ml12-lg-2 ml12-m-3 ml12-s-4 ml12-xs-5 semi-attached-table"}))
        df_table = pd.read_html(tabela)
        #concatenando os dataframe
        df_Aux = pd.concat([df_Aux,df_table[item].drop("Player",axis=1).rename({"Player.1":"Player"},axis=1)],ignore_index=True)        
        #clicando no link next para ir para a proxima pagina
        button_next = driver.find_elements(By.CSS_SELECTOR,"a#next.option.clickable")[item]
        wait.until(EC.element_to_be_clickable((button_next)))        
        driver.find_elements(By.CSS_SELECTOR,"a#next.option.clickable")[item].click()
        time.sleep(5)
    #leitura das tabelas
    html = driver.page_source
    Beautiful_Soup = BeautifulSoup(html, 'html.parser')
    tabela = str(Beautiful_Soup.find_all('div', {'class': "ml12-lg-2 ml12-m-3 ml12-s-4 ml12-xs-5 semi-attached-table"}))
    df_table = pd.read_html(tabela)
    #concatenando os dataframe dependendo do link que esta
    df_Aux = pd.concat([df_Aux,df_table[item].drop("Player",axis=1).rename({"Player.1":"Player"},axis=1)],ignore_index=True)    
    if item == 4:
        df_Player_Summary = pd.concat([df_Player_Summary,df_Aux],ignore_index=True)
    elif item == 5:
        df_Player_Defensive = pd.concat([df_Player_Defensive,df_Aux],ignore_index=True)         
    elif item == 6:
        df_Player_Offensive = pd.concat([df_Player_Offensive,df_Aux],ignore_index=True)         
    elif item == 7:
        df_Player_Passing = pd.concat([df_Player_Passing,df_Aux],ignore_index=True)
    #clicando em preview e em first para que o voltar no loop dos links
    driver.find_elements(By.CSS_SELECTOR,"a#prev.option.clickable")[0].click()
    time.sleep(2)
    driver.find_elements(By.CSS_SELECTOR,"a#first.option.clickable")[0].click()
    driver.find_element(By.XPATH,list_link_player[item-4]).click()

In [14]:
#Declaração de variaveis
tpages = driver.find_elements(By.ID,"totalPages")
df_Player_Detailed = pd.DataFrame()
wait = WebDriverWait(driver,timeout=10)
item = -1
driver.find_element(By.XPATH,list_link_player[-2]).click()
df_Aux = pd.DataFrame()
tpages = driver.find_elements(By.ID,"totalPages")
#loop que vai mudar de pagina
for page in range(1,int(tpages[item].get_attribute(name='value'))):
    #leitura das tabelas
    html = driver.page_source
    Beautiful_Soup = BeautifulSoup(html, 'html.parser')
    tabela = str(Beautiful_Soup.find_all('div', {'class': "ml12-lg-2 ml12-m-3 ml12-s-4 ml12-xs-5 semi-attached-table"}))
    df_table = pd.read_html(tabela)
    #concatenando os dataframe, tirando a coluna Player e renomeando a Player.1 para Player
    df_Aux = pd.concat([df_Aux,df_table[item].drop("Player",axis=1).rename({"Player.1":"Player"},axis=1)],ignore_index=True)        
    #clicando no link next para ir para a proxima pagina
    button_next = driver.find_elements(By.CSS_SELECTOR,"a#next.option.clickable")[item]
    wait.until(EC.element_to_be_clickable((button_next)))        
    driver.find_elements(By.CSS_SELECTOR,"a#next.option.clickable")[item].click()
    time.sleep(7)
#leitura das tabelas
html = driver.page_source
Beautiful_Soup = BeautifulSoup(html, 'html.parser')
tabela = str(Beautiful_Soup.find_all('div', {'class': "ml12-lg-2 ml12-m-3 ml12-s-4 ml12-xs-5 semi-attached-table"}))
df_table = pd.read_html(tabela)
#concatenando os dataframe dependendo do link que esta
df_Aux = pd.concat([df_Aux,df_table[item].drop("Player",axis=1).rename({"Player.1":"Player"},axis=1)],ignore_index=True)    
df_Player_Detailed = pd.concat([df_Player_Detailed,df_Aux],ignore_index=True)
#clicando em preview e em first para que o voltar no loop dos links
driver.find_elements(By.CSS_SELECTOR,"a#prev.option.clickable")[0].click()
time.sleep(2)
driver.find_elements(By.CSS_SELECTOR,"a#first.option.clickable")[0].click()

In [15]:
df_Player_Summary

,Player,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,29,1,2,-,5.0,77.1,1.7,7,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",24(1),2132,15,10,7,-,4.0,82.0,0.1,8,7.89
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,20,10,1,-,4.3,80.2,0.2,6,7.77
3,"Erling Haaland Borussia Dortmund, 21, FW",13(2),1172,16,5,3,-,3.9,70.4,1.8,4,7.76
4,"Karim Benzema Real Madrid, 34, FW",24,1974,20,10,-,-,3.8,85.6,0.6,9,7.71
...,...,...,...,...,...,...,...,...,...,...,...,...
1565,"Jonathan Iglesias Clermont Foot, 33, DMC",5(10),571,-,-,-,-,0.1,88.6,0.1,-,6.12
1566,"Kevin Möhwald Union Berlin, 28, M(CR)",2(12),286,-,-,2,-,0.6,82.2,0.1,-,6.12
1567,"M'Baye Niang Bordeaux, 27, AM(LR),FW",1(14),299,1,-,3,-,1.1,69.6,0.6,-,6.11
1568,"Alejandro Cantero Levante, 21, Midfielder",2(16),345,-,1,1,-,0.4,68.3,0.1,-,6.11


In [16]:
df_Player_Offensive

,Player,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,29,1,5.0,1.2,1.5,1.1,0.7,1.3,2.4,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",24(1),2132,15,10,4.0,1.9,3.3,1.5,1.1,1.8,3.2,7.89
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,20,10,4.3,2.0,1.8,0.6,0.5,2.0,3.0,7.77
3,"Erling Haaland Borussia Dortmund, 21, FW",13(2),1172,16,5,3.9,0.9,0.7,0.7,0.5,0.6,2.8,7.76
4,"Karim Benzema Real Madrid, 34, FW",24,1974,20,10,3.8,1.9,1.0,0.3,1.1,0.7,1.3,7.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,"Jonathan Iglesias Clermont Foot, 33, DMC",5(10),571,-,-,0.1,0.3,0.1,0.7,-,0.4,0.4,6.12
1566,"Kevin Möhwald Union Berlin, 28, M(CR)",2(12),286,-,-,0.6,0.2,0.1,0.1,0.1,0.3,0.4,6.12
1567,"M'Baye Niang Bordeaux, 27, AM(LR),FW",1(14),299,1,-,1.1,0.1,-,0.3,0.1,0.5,0.8,6.11
1568,"Alejandro Cantero Levante, 21, Midfielder",2(16),345,-,1,0.4,0.1,0.4,0.2,-,0.2,0.6,6.11


In [17]:
df_Player_Defensive

,Player,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,0.3,0.1,0.6,0.1,0.3,0.3,0.1,-,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",24(1),2132,0.3,0.2,0.9,-,0.4,0.2,-,-,7.89
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,0.4,0.2,0.3,-,0.2,0.4,-,-,7.77
3,"Erling Haaland Borussia Dortmund, 21, FW",13(2),1172,-,0.1,0.7,0.1,1.3,0.1,0.1,-,7.76
4,"Karim Benzema Real Madrid, 34, FW",24,1974,0.5,0.2,0.5,-,0.4,0.3,-,-,7.71
...,...,...,...,...,...,...,...,...,...,...,...,...
1565,"Jonathan Iglesias Clermont Foot, 33, DMC",5(10),571,1.2,0.3,0.7,-,0.1,0.5,0.1,-,6.12
1566,"Kevin Möhwald Union Berlin, 28, M(CR)",2(12),286,0.4,0.1,0.4,-,0.4,0.4,-,-,6.12
1567,"M'Baye Niang Bordeaux, 27, AM(LR),FW",1(14),299,0.1,-,0.9,0.1,0.1,0.1,0.1,-,6.11
1568,"Alejandro Cantero Levante, 21, Midfielder",2(16),345,0.1,0.1,0.6,-,-,-,-,-,6.11


In [18]:
df_Player_Passing

,Player,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,1,1.2,21.5,77.1,0.2,0.5,0.2,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",24(1),2132,10,1.9,35.3,82.0,0.7,1.2,-,7.89
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,10,2.0,33.4,80.2,0.2,0.4,0.3,7.77
3,"Erling Haaland Borussia Dortmund, 21, FW",13(2),1172,5,0.9,19.6,70.4,-,-,0.1,7.76
4,"Karim Benzema Real Madrid, 34, FW",24,1974,10,1.9,38.0,85.6,0.1,0.9,0.1,7.71
...,...,...,...,...,...,...,...,...,...,...,...
1565,"Jonathan Iglesias Clermont Foot, 33, DMC",5(10),571,-,0.3,24.1,88.6,-,0.7,0.1,6.12
1566,"Kevin Möhwald Union Berlin, 28, M(CR)",2(12),286,-,0.2,11.2,82.2,0.1,0.4,-,6.12
1567,"M'Baye Niang Bordeaux, 27, AM(LR),FW",1(14),299,-,0.1,4.6,69.6,-,0.1,0.1,6.11
1568,"Alejandro Cantero Levante, 21, Midfielder",2(16),345,1,0.1,3.5,68.3,0.1,0.2,-,6.11


In [19]:
df_Player_Detailed

,Player,Apps,Mins,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,"Mattia Zaccagni Verona, 26, AM(CL),FW",2,180,2,-,-,2,8.47
1,"Robert Lewandowski Bayern Munich, 33, FW",26,2261,5,0.8,0.8,3.4,8.03
2,"Kylian Mbappé Paris Saint-Germain, 23, AM(LR),FW",25,2132,4,1,0.2,2.8,7.89
3,"Kieran Trippier Newcastle United, 31, D(R),M(R)",4,318,0.5,0.5,-,-,7.87
4,"Andy Delort Montpellier, 30, AM(R),FW",3,270,4.7,1.3,0.7,2.7,7.82
...,...,...,...,...,...,...,...,...
2797,"Maxwel Cornet Lyon, 25, D(L),M(LR),FW",1,65,-,-,-,-,5.06
2798,"Juan Pérez Osasuna, 25, GK",1,90,-,-,-,-,5.01
2799,"Gianluca Pegolo Sassuolo, 40, GK",1,90,-,-,-,-,5.00
2800,"Joan García Espanyol, 20, Goalkeeper",2,107,-,-,-,-,5.00


# Parte 2

In [20]:
#clicando em Brasileirão
driver.find_element(By.CSS_SELECTOR,"#popular-tournaments-list > li:nth-child(9) > a").click()

In [21]:
#selecionando o ano de 2021, a tabela de 2022 ainda não começou e não há tabela 
Select(driver.find_element(By.NAME,"seasons")).select_by_visible_text("2021")

In [34]:
#faz a leitura da tabela e retira os numeros que aparece antes dos nomes dos times
Brasileirao,bs = Read_Table_Statistics(driver,"standings-19551-grid")


In [29]:
Brasileirao["Team"] = Brasileirao["Team"].str.split("\d+",1,expand = True)[1]

TypeError: list indices must be integers or slices, not str

In [38]:
Brasileirao["Team"].str.split(" ",1, expand=True)

TypeError: list indices must be integers or slices, not str

In [35]:
Brasileirao

[                      Team   P   W   D   L  GF  GA  GD  Pts    Form
 0             1Atletico MG  38  26   6   6  67  34  33   84  wdwwwl
 1                2Flamengo  38  21   8   9  69  36  33   71  wdwdll
 2               3Palmeiras  38  20   6  12  58  43  15   66  lldwdw
 3               4Fortaleza  38  17   7  14  44  45  -1   58  lwlwlw
 4             5Corinthians  38  15  12  11  40  36   4   57  lwlwdl
 5     6Red Bull Bragantino  38  14  14  10  55  46   9   56  wldllw
 6              7Fluminense  38  15   9  14  38  38   0   54  lwwllw
 7              8America MG  38  13  14  11  41  37   4   53  dldwdw
 8             9Atletico GO  38  13  14  11  33  36  -3   53  ddwwww
 9              10Santos FC  38  12  14  12  35  40  -5   50  wlwdwd
 10                 11Ceara  38  11  17  10  39  38   1   50  wdwldl
 11         12Internacional  38  12  12  14  44  42   2   48  llldll
 12             13Sao Paulo  38  11  15  12  31  39  -8   48  wdwlwl
 13  14Athletico Paranaense  38  1

# Parte 3

In [ ]:
#Escreve Chelsea na pesquisa
driver.find_element(By.ID,"search-box").send_keys("Chelsea"+Keys.RETURN)

In [ ]:
#clica no link com o texto de Chelsea
driver.find_element(By.PARTIAL_LINK_TEXT,"Chelsea").click()

In [ ]:
#clica em Offensive
driver.find_element(By.CSS_SELECTOR,"#team-squad-stats-options > li:nth-child(3)").click()

In [ ]:
#leitura da tabela de dados
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
tabela = str(soup.find_all('div', {'id': "statistics-table-offensive"}))
df = pd.read_html(tabela)

In [ ]:
#um dataframe com diversar tabela, mas que interessa está em 0
ChampionLeague = df[0]

In [ ]:
#Retira a coluna Player, renomeia a coluna Player.1 para Player
ChampionLeague = ChampionLeague.head(5).drop("Player",axis=1).rename({"Player.1":"Player"},axis=1)

In [ ]:
ChampionLeague